In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import dos modulos que usarei para a classificação

#!pip install category_encoders


import pandas as pd
from category_encoders import *
from category_encoders import CountEncoder
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
#Import dos dados

df_test = pd.read_csv('../input/mini-flight-delay-prediction/flight_delays_test.csv')
df_train = pd.read_csv('../input/mini-flight-delay-prediction/flight_delays_train.csv')

df_train.info()

In [ ]:
#Separando as 'respostas' do dataframe para poder usa-las no treinamento mais tarde
label = df_train['dep_delayed_15min']
df_train = df_train.drop('dep_delayed_15min',axis = 1)
label = pd.get_dummies(label)
label = label['N'].values #Nesse momento transformamos nossas respostas que estão 'S' ou 'N' para binario
label

In [ ]:
#Aqui nos juntamos os dataframes de teste e de treinamento em um so dataframe para poder treinar nosso encoder e 'transformar' os dados nominais em numericos

df = pd.concat([df_train,df_test])

dest_y_train = df_train['Dest']
uc_y_train = df_train['UniqueCarrier']
day_y_train = df_train['DayofMonth']

enc_aerporto = CountEncoder(cols=['Dest', 'Origin']).fit(df, dest_y_train)
enc_uc = CountEncoder(cols=['UniqueCarrier']).fit(df, uc_y_train)
enc_day = CountEncoder(cols=['Month','DayofMonth','DayOfWeek']).fit(df, day_y_train) #decidi criar um encoder para as colunas de dias apenas para que o df 'padrao'

In [ ]:
#Usamos o encoder treinado para deixar os dataframes pronto para a classificação
df_train_num = enc_aerporto.transform(df_train)
df_train_num = enc_uc.transform(df_train_num)
df_train_num = enc_day.transform(df_train_num)

df_test_num = enc_aerporto.transform(df_test)
df_test_num = enc_uc.transform(df_test_num)
df_test_num = enc_day.transform(df_test_num)

df_train_num.head()
#df_train_num.info()

In [ ]:
#Normalização dos df
x = df_train_num.values

min_max_scaler = preprocessing.MinMaxScaler()
xscaled = min_max_scaler.fit_transform(x)
df_normalized = pd.DataFrame(xscaled)
df_normalized.head()

In [ ]:
#Nos vamos separar nossos dados de treinamento para a classificação pois não temos o label dos dados de teste
X = df_normalized.values
y = label
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.01,random_state=42)

In [ ]:
#model train
mlp = MLPClassifier(hidden_layer_sizes=(500, 20),random_state=1, max_iter=300)
mlp.fit(X_train,y_train)

In [ ]:
y_pred_train = mlp.predict(X_test)
print('F1-score: ', metrics.f1_score(y_test, y_pred_train))

In [ ]:
y_pred_test = mlp.predict(df_test_num.values)

label_test = []

for predict in y_pred_test:
  if predict == 0:
    label_test.append("N")
  else:
    label_test.append("Y")

df_label = pd.DataFrame(label_test)
df_label.to_csv('submission.csv', index=False)